In [1]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

Using TensorFlow backend.


# Move Explanations
The first step is to prepare the data.
I already created a dataset in an earlier post.
However, I no wknow that I need a different format so I will have to redo some of the steps.

## Datasets with annotation
Next, I will look at the ~4000 games with annotations sourced from http://www.angelfire.com/games3/smartbridge/

The below command will throw errors but I handle these (they are still displayed even though they are irrelevant).

In [2]:
from os import listdir
from tqdm import tqdm
from chess import pgn
import chess

annotated_games = []

for file in tqdm(listdir('data/annotated')):
    pgn = open('data/annotated/' + file)
    while True:
        game = chess.pgn.read_game(pgn)
        
        if game is None:
            break
            
        if len(game.errors) > 0:
            continue
        
        annotated_games.append(game)

  3%|▎         | 1/38 [00:00<00:04,  7.83it/s]error during pgn parsing
Traceback (most recent call last):
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'g1-f3' in rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
error during pgn parsing
Traceback (most recent call last):
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "C:\Users\Robin\Anaconda3\lib\site-pack

ValueError: illegal san: 'Bxg5' in rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
 32%|███▏      | 12/38 [00:03<00:05,  4.38it/s]error during pgn parsing
Traceback (most recent call last):
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\pgn.py", line 709, in parse_san
    return board.parse_san(san)
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\__init__.py", line 2739, in parse_san
    raise ValueError("ambiguous san: {!r} in {}".format(san, self.fen()))
ValueError: ambiguous san: 'Nd7' in rnbqk2r/ppp1bppp/4pn2/3p2B1/2PP4/2N2N2/PP2PPPP/R2QKB1R b KQkq - 5 5
error during pgn parsing
Traceback (most recent call last):
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\pgn.py", line 1271, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "C:\Users\Robin\Anaconda3\lib\site-packages\chess\pgn.

In [3]:
len(annotated_games)

3683

In [4]:
for moves in annotated_games[0].mainline():
    print(str(moves.move) + " " + moves.comment)

e2e4 
c7c5 Although Shirov has played a variety of defenses, I think he's been playing the Petroff's Defense and perhaps the Caro-Kann most recently.
g1f3 
b8c6 
d2d4 
c5d4 
f3d4 
e7e5 Neo-Sveshnikov or Lowenthal leaves d5 (and perhaps d6) weak. More common is 4...Nf6, forcing White to defend Pe4 and threatening ...d5.
d4b5 
d7d6 
b1c3 He threatens Nc3-d5 and Nc7+, so Black must nip that idea in the bud.
a7a6 
b5a3 
b7b5 Instead of White dominating square c4 it's Black who controls it, for the moment. Eventually piece play should dominate a position more than pawns. There is also the immediate threat of ...b4.
c3d5 White could continue with Bc1-e3-b6 and Nc7+, so Black has to react quickly.
c6e7 
c2c4 
e7d5 
e4d5 This makes the pawns a bit unbalanced with White advancing on the queen-side. Aside from Pb5 Black shouldn't have much difficulty completing his development.
b5c4 
a3c4 Now White controls c4!
g8f6 
c1e3 
a8b8 
f1e2 
f8e7 
a2a4 aiming for a4-a5 and Nb6 or Bb6 to cramp Black, bu

In [5]:
import re

def dataExtractor2(games):
    """
    Extracts the moves and fens of each game as strings
    path - the path to the pgn file in which the games are stored
    returns an array of objects containing the moves and the fens
    """
    
    counter = 0
    
    re0 = re.compile(r"{.*?}", re.MULTILINE)
    re1 = re.compile(r"{.*}", re.MULTILINE)
    re2 = re.compile(r"\d+\..", re.MULTILINE)
    re3 = re.compile(r"\.", re.MULTILINE)
    re4 = re.compile(r"\$\d+", re.MULTILINE)
    
    data = []
    for game in tqdm(games):
        result = [] # Stores all the commented fens for this game
        
        string = str(game.mainline_moves())
        string = re.sub(re0, "", string) # Remove comments
        string = re.sub(re1, "", string)
        string = re.sub(re2, "", string)
        string = re.sub(re3, "", string)
        string = re.sub(re4, "", string)
        
        if "{" in string:
            continue
            
        white = [name.strip(' ') for name in game.headers["White"].split(",")]
        black = [name.strip(' ') for name in game.headers["Black"].split(",")]
        
        moves = list(filter(None, string.split(" ")))
        mainline = list(game.mainline())
            
        board = game.board()                
        
        for i in range(len(moves)):
            move = moves[i]
            comment = mainline[i].comment
            board_prev = str(board.board_fen())
            board.push(mainline[i].move)
            board_after = str(board.board_fen())
            
            if comment: # comment exists  
                counter += 1
                for name in white:
                    comment = comment.replace(name, 'white')
                for name in black:
                    comment = comment.replace(name, 'black')
                result.append([[board_prev, board_after], comment])
            
        data.append(result)
                               
    print(counter, 'commented moves found')        
    return data
                               
data = dataExtractor2(annotated_games)

100%|██████████| 3683/3683 [00:33<00:00, 111.31it/s]


17615 commented moves found


In [6]:
data[0][0]

[['rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR',
  'rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR'],
 "Although black has played a variety of defenses, I think he's been playing the Petroff's Defense and perhaps the Caro-Kann most recently."]

## Caption Generation

In [214]:
# Helper methods to convert fens to the matrix representation
import numpy as np

def indexToArray(i, len_ = 13):
    '''
    Converts an index into a one-hot-encoded vector.
    i - the index of the 1.
    len_ - (optional) the len of the one-hot-vector. Default is 12.
    returns a vector of length len_
    '''
    
    array = [0] * len_
    
    if(i >= 0 and i < len(array)):
        array[i] = 1
        
    return array

def fenToMatrix(fen):
    '''
    Converts a fen string to a 8x8x16 matrix.
    fen - a string in the fen notation
    returns a 8x8x12 matrix
    '''
    
    # 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR'
    pieces = {
        '': indexToArray(0),
        'r': indexToArray(1),
        'n': indexToArray(2),
        'b': indexToArray(3),
        'q': indexToArray(4),
        'k': indexToArray(5),
        'p': indexToArray(6),
        'P': indexToArray(7),
        'R': indexToArray(8),
        'N': indexToArray(9),
        'B': indexToArray(10),
        'Q': indexToArray(11),
        'K': indexToArray(12),
    }
    
    matrix = []
    row = []
    
    for c in fen:
        try:
            cInt = int(c)
            
            for i in range(cInt):
                row.append(pieces[''])
        except: # c can not be cast as integer
            if c == '/':
                matrix.append(row)
                row = []
            else:
                row.append(pieces[c]) 
    matrix.append(row)
                
    return np.array(matrix)

def matrixToFen(matrix):
    '''
    Converts 8x8x13 matrix to a fen string.
    matrix - a 8x8x13 matrix
    returns a fen string
    '''
    
    # 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR'
    pieces = [
        '',
        'r',
        'n',
        'b',
        'q',
        'k',
        'p',
        'P',
        'R',
        'N',
        'B',
        'Q',
        'K',
    ]
        
    matrix = (matrix > 0).astype(int) * (matrix == matrix.max(axis=2)[:,:,None]).astype(int)
        
    fen = ""
     
    for i in range(8):
        row = matrix[i]
        empty = 0
        
        for j in range(8):
            square = row[j]
            piece_found = False
            
            for k in range(len(pieces)):
                piece = square[k]
                
                if piece == 1:
                    if pieces[k] == '':
                        empty += 1
                    else:
                        if empty > 0:
                            fen += str(empty)
                            empty = 0
                        fen += pieces[k]               
                
        if empty > 0:
            fen += str(empty)  
            
        if i < 7:
            fen += "/"  
            
    return fen

def similarityScore(y_true, y_pred):
    '''
    Calculates the similarity between to chess position
    The result is the number of squares that have the same piece/are empty accross the two boards
    y_true - an 8x8x12 matrix
    y_pred - and 8x8x12 matrix
    returns a similarity between 0 and 1
    '''
    
    score = 0
    
    y_true = (y_true > 0).astype(int) * (y_true == y_true.max(axis=2)[:,:,None]).astype(int)
    y_pred = (y_pred > 0).astype(int) * (y_pred == y_pred.max(axis=2)[:,:,None]).astype(int)
    
    diff = (y_true - y_pred) ** 2
    
    wrong = np.sum(diff) 
    
    return 1 - (wrong / 32)  # Two boards can at most have 32 different fields (or they can have 16 fields where different units are places which also results in a wrong-count of 32) 
    
print(data[0][0][0][0])
print(matrixToFen(fenToMatrix(data[0][0][0][0])))

r1bqkbnr/pp1ppppp/2n5/8/3NP3/8/PPP2PPP/RNBQKB1R
r1bqkbnr/pp1ppppp/2n5/8/3NP3/8/PPP2PPP/RNBQKB1R


Creating a volcabular

In [8]:
vocab = {
    "<Pad>": 0,
    "<End>": 1
}

for game in tqdm(data):
    for fens, caption in game:
        words = caption.split(' ')
        
        for word in words:
            word = word.lower()
            if word not in vocab:
                vocab[word] = len(vocab)
                
print(len(vocab))

100%|██████████| 2207/2207 [00:00<00:00, 22278.39it/s]


25330


Next I need to split the captions into words with one target word

In [9]:
data_preped = []
max_ = 0

for game in tqdm(data):
    for fens, caption in game:
        words = caption.split(' ')
        result = []
        
        for word in words:
            word = word.lower()            
            data_preped.append([fens, result, word])  
            result.append(word)
            
        max_ = max(max_, len(result) + 1)
            
        data_preped.append([fens, result, "<End>"])  
                
print("Data", len(data_preped))
print("Longest caption", max_)

100%|██████████| 2207/2207 [00:00<00:00, 2870.76it/s]


Data 341278
Longest caption 277


Now I create a data generator

In [10]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, vocab):
        'Initialization'
        self.batch_size = 64
        self.data = data
        self.vocab = vocab

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        data = self.data[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(data)

        return X, y

    def __data_generation(self, data):
        'Generates data containing batch_size samples' 
        from_matrices = []
        to_matrices = []
        sentences = []
        targets = []

        for fens, sentence, t in data:
            from_matrix = fenToMatrix(fens[0])
            to_matrix = fenToMatrix(fens[1])
            
            from_matrices.append(from_matrix)
            to_matrices.append(to_matrix)
            
            sentence = [self.vocab[word] for word in sentence]
            
            target = np.zeros(len(self.vocab))
            target[self.vocab[t]] = 1
            sentence = sentence + [0] * (277 - len(sentence)) # Add padding

            sentences.append(sentence)
            targets.append(target)
            
        from_matrices = np.array(from_matrices)
        to_matrices = np.array(to_matrices)
        sentences = np.array(sentences)
        targets = np.array(targets)
            
        return [from_matrices, to_matrices, sentences], targets
    
np.random.shuffle(data_preped)
training_generator = DataGenerator(data_preped[:330000], vocab)
validation_generator = DataGenerator(data_preped[330000:], vocab)

In [11]:
training_generator[0][1][0].shape

(25330,)

In [12]:
from keras.models import load_model

model = load_model('autoencoder1.h5')
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8, 8, 13)          0         
_________________________________________________________________
model_2 (Model)              (None, 256)               1509120   
_________________________________________________________________
model_1 (Model)              (None, 8, 8, 13)          1509696   
Total params: 3,018,816
Trainable params: 3,018,816
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
from keras.models import Model
from keras.layers import Dense, Activation, Input, Flatten, Reshape, GRU, Embedding, concatenate
from keras import metrics

def Encoder():
    from_matrix = Input(shape=(8, 8, 13))
    to_matrix = Input(shape=(8, 8, 13))
    sentence = Input(shape=(10,))
    
    encoder = load_model('autoencoder1.h5').get_layer('model_2')
    encoder.name = 'encoder1'
    encoder.layers.pop(0)
    x1 = encoder(from_matrix)
    x1 = Model(inputs=from_matrix, outputs=x1)
    
    # Freeze the encoder
    for layer in x1.layers:
        layer.trainable=False
    
    encoder = load_model('autoencoder1.h5').get_layer('model_2')
    encoder.name = 'encoder2'
    encoder.layers.pop(0)
    x2 = encoder(to_matrix)
    x2 = Model(inputs=to_matrix, outputs=x2)
    
    # Freeze the encoder
    for layer in x2.layers:
        layer.trainable=False

    y = Embedding(len(vocab), 150, mask_zero=True)(sentence)
    y = GRU(124, activation='relu')(y)
    y = Model(inputs=sentence, outputs=y)
    
    z = concatenate([x1.output, x2.output, y.output])
    
    return Model(inputs=[x1.input, x2.input, y.input], outputs=z)

def Decoder():
    X = Input(shape=(636,))
    d1 = Dense(256, activation='relu')(X)
    d2 = Dense(len(vocab), activation='softmax')(d1)
    return Model(X, d2)


# define input to the model:
X = [Input(shape=(8, 8, 13)), Input(shape=(8, 8, 13)), Input(shape=(None,))]

# make the model:
explanation_model = Model(X, Decoder()(Encoder()(X)))

# compile the model:
explanation_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy])

In [14]:
explanation_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8, 8, 13)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 8, 8, 13)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 636)          6920040     input_1[0][0]                    
                                                                 input_2[0][0]                    
          

In [17]:
explanation_model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=5)

Epoch 1/5
5156/5156 [==============================] - 2793s 542ms/step - loss: 6.8795 - categorical_accuracy: 0.0699 - val_loss: 6.8913 - val_categorical_accuracy: 0.0728
Epoch 2/5
5156/5156 [==============================] - 2403s 466ms/step - loss: 6.6681 - categorical_accuracy: 0.0737 - val_loss: 6.7537 - val_categorical_accuracy: 0.0741
Epoch 3/5
5156/5156 [==============================] - 2482s 481ms/step - loss: 6.5189 - categorical_accuracy: 0.0764 - val_loss: 6.7392 - val_categorical_accuracy: 0.0740
Epoch 4/5
5156/5156 [==============================] - 2524s 489ms/step - loss: 6.4150 - categorical_accuracy: 0.0774 - val_loss: 6.7313 - val_categorical_accuracy: 0.0760
Epoch 5/5
5156/5156 [==============================] - 2547s 494ms/step - loss: 6.3417 - categorical_accuracy: 0.0768 - val_loss: 6.7344 - val_categorical_accuracy: 0.0774


## Part 2 - Transforming explanations

In [10]:
# Lets take a closer look at the explanations

explanations = []

for game in tqdm(data):
    for fens, caption in game:
        explanations.append(caption)

100%|██████████| 2207/2207 [00:00<00:00, 602070.17it/s]


In [11]:
import nltk
from nltk.corpus import stopwords

frequency = {}
stop_words = set(stopwords.words('english'))

for explanation in tqdm(explanations):
    words = nltk.word_tokenize(explanation)
    
    for word in words:
        word = word.lower()
        if word not in stop_words and len(word) > 1:
            if word not in frequency:
                frequency[word] = 1
            else:
                frequency[word] += 1
len(frequency)

100%|██████████| 17615/17615 [00:03<00:00, 4651.79it/s]


15201

We have 15201 words, lets look at the most frequent ones

In [14]:
import operator

sorted_frequency = sorted(frequency.items(), key=lambda kv: kv[1], reverse=True)
sorted_frequency

[('black', 9732),
 ('white', 9285),
 ("'s", 5465),
 ('...', 4338),
 ('without', 3989),
 ('pawn', 3031),
 ('knight', 2058),
 ('move', 2055),
 ('bishop', 2015),
 ('king', 1822),
 ('queen', 1752),
 ("n't", 1461),
 ('rook', 1314),
 ('would', 1308),
 ('position', 1200),
 ('attack', 1191),
 ('play', 1078),
 ('pieces', 995),
 ('center', 968),
 ('game', 964),
 ('pawns', 962),
 ('kingside', 803),
 ('time', 762),
 ('square', 651),
 ('one', 642),
 ('better', 638),
 ('open', 631),
 ('advantage', 579),
 ('also', 575),
 ('piece', 561),
 ('moves', 561),
 ('threat', 555),
 ('way', 544),
 ('queenside', 544),
 ('good', 524),
 ('two', 524),
 ('back', 519),
 ('win', 506),
 ('see', 503),
 ('advance', 488),
 ('since', 485),
 ('idea', 476),
 ('get', 473),
 ('control', 471),
 ('defense', 462),
 ('could', 461),
 ('diagonal', 453),
 ('material', 447),
 ('exchange', 442),
 ('development', 428),
 ('pressure', 420),
 ('still', 408),
 ('mate', 405),
 ('d5', 403),
 ('squares', 398),
 ('must', 396),
 ('played', 394),

In [88]:
import re

def filterExplanations(reg):
    return [explanation for explanation in explanations if re.search(reg, explanation)]

In [132]:
# Now lets look at the most common words and see if we cant find a pattern

# The first thing I want to look at is the idea of threat.
filterExplanations("threat")

['Neo-Sveshnikov or Lowenthal leaves d5 (and perhaps d6) weak. More common is 4...Nf6, forcing White to defend Pe4 and threatening ...d5.',
 'He threatens Nc3-d5 and Nc7+, so Black must nip that idea in the bud.',
 "Instead of White dominating square c4 it's Black who controls it, for the moment. Eventually piece play should dominate a position more than pawns. There is also the immediate threat of ...b4.",
 'threatening ...f4 and ...e5-e4-e3, but once that threat is avoided then what is Bf6 doing?',
 'defending Pe4 and threatening Be2-b5 or a5-a6 seems correct',
 "Now begins a forcing sequence. White doesn't want to move Rb4, so he threatens Black.",
 'Now what? Be8 and Pa5 are threatened. Obviously Be8 needs to move and threaten something or White needs to threaten Bb4 or Kg8.',
 "After this it's difficult to find a good move for Black, despite there being no direct threat.",
 "He might consider capturing Nc5 in some circumstances, but it appears likely he's repositioning the bishop 

In [206]:
# Often a sentence with threat also contains concrete squares
# Lets try to extract these

def threatExtractor(explanation, max_=1):
    sentences = nltk.sent_tokenize(explanation)
    
    for sentence in sentences:
        if not re.search("threat", sentence):
            continue
        
        moves = re.findall("threat.*?([A-Za-z]+[0-9]+)[ |\.]", explanation)
        
        if len(moves) == 0:
            continue
            
        moves = moves[:max_]
        
        result = "This move threatens " + ", ".join(moves) + "."
        return result
    return None

threats = [threatExtractor(explanation) for explanation in explanations if threatExtractor(explanation)]
len(threats)

667

In [222]:
# Next, I look at mistakes
filterExplanations("(mistake|blunder)")

['threatening ...Nxf2 and trying to divert Qf3, so he can capture Nc3 to "own" the diagonal and, in particular, d4. It\'s hard to criticize this move because it is done with good intentions, but white\'s reply does make it appear to be a mistake. It\'s just terribly unobvious.',
 "This move might be a mistake except for the fact that in the endgame it's alright to trade the king-side pawns for Black's queen-side pawns. The better-formed queen-side pawns, if unblocked, are better. It's a curious thing also that there are times when a rook and knight work better together than a rook and bishop, so Black's Bg7 might be valued highly at the moment, but only a few moves from now the knight could be better!",
 'But this is a serious mistake. White can now open the a2-g8 diagonal and the tactics will be awful for Black.',
 'This move is a mistake as White does not need to broaden his pawn chain especially at the expense of his development. Better was 5.Nf3. 5.Nf3 Qb6 6.Be2 Bd7 7.a3 c4',
 'Bla

In [223]:
def mistakeExtractor(explanation):
    if re.search("(mistake|blunder)", explanation):
        return "This move is a mistake."
    
    return None

mistakes = [mistakeExtractor(explanation) for explanation in explanations if mistakeExtractor(explanation)]
len(mistakes)

209

In [176]:
# What about good moves
filterExplanations("good")
# The results seem to varied, not easy to parse

["After this it's difficult to find a good move for Black, despite there being no direct threat.",
 "black would rather prepare O-O than try to develop Bc8 and Nb8. His Be7 isn't very good, but once developed his two bishops should blossom.",
 "black did a good job of keeping his king safe, but he's down an exchange and Pb4 is en prise. His only trump is that Pf6 is passed.",
 'Black has given up a pawn, but he still has the basic elements of a good position: safe king, open lines for his pieces and no more weak pawns than his opponent.',
 "Now it's just transposed to a King's Indian Defense, Fianchetto Variation which is still considered good for White, though the players involved make the difference. In this case black as Black should be up to the task.",
 'She chose a good moment for this advance; after Black had moved toward the queen-side and weakened his control of e5. It might have also been a 40-move time control.',
 "White is in good control of the diagonal, but can't O-O-O. B

In [177]:
# Now lets look at attacks
filterExplanations("attack")

["Pd5 is poison because of Bxh7+, but if Black captures Pb5 he might create some play on the b-file while trading-off White's better attacking pieces (Bd3 for starters) .",
 "Pd5 has done it's job, holding Black down, so the attack on Kh8 proceeds and with great force.",
 'This could be to free Bg2 or to threaten h4-h5xg6 to create an attack on the h-file. At this point it might seem to White that trading pieces (such as Rc1-a1) is risky because there are several weak pawns which need defenders.',
 'winning a piece and the game! That was a pretty neat accurate forceful attack and defense. It shows how careful one has to be with their king to avoid problems.',
 "Quickly, after the queen trade we're in an ending and Black's better pawns are good, but not nearly enough to convey an advantage. The opposite color bishops don't mean much at this point. If one side was to build up an attack it could be extra useful, but more likely the position will lead to simplifying rook trades and the bis

In [199]:
def attackExtractor(explanation, max_=1):
    sentences = nltk.sent_tokenize(explanation)
    
    for sentence in sentences:
        if not re.search("attack", sentence):
            continue
        
        moves = re.findall("attack.*?([A-Za-z]+[0-9]+)[ |\.]", explanation)
        
        if len(moves) == 0:
            continue
            
        moves = moves[:max_]
        
        result = "This move attacks " + ", ".join(moves) + "."
        return result
    return None

attacks = [attackExtractor(explanation) for explanation in explanations if attackExtractor(explanation)]
len(attacks)

428

In [179]:
filterExplanations("(protect)|(defend)")

['Neo-Sveshnikov or Lowenthal leaves d5 (and perhaps d6) weak. More common is 4...Nf6, forcing White to defend Pe4 and threatening ...d5.',
 "aiming for a4-a5 and Nb6 or Bb6 to cramp Black, but making Pb2 backward! That and the slightly exposed Pd5 make White's position suspicous. Where's his offense coming from? I suppose he thinks it's necessary to use a Nb6 to defend Pd5, but ...Nf6-d7 can chase a Nb6 away, exposing Pb2. If he continues his queen-side advance then one of the pawns gives him the advantage. So, where's Black's offense come from?",
 "White doesn't want to play f3-f4 and leave Black with a protected passed pawn at e4. And, I understand that feeling. However, to open lines means to give Black other possibilities.",
 'defending Pe4 and threatening Be2-b5 or a5-a6 seems correct',
 "He might consider capturing Nc5 in some circumstances, but it appears likely he's repositioning the bishop to d4. It's not desirable to spend an extra tempo for this kind of thing, but the Bf4 j

In [200]:
def defenseExtractor(explanation, max_=1):
    sentences = nltk.sent_tokenize(explanation)
    
    for sentence in sentences:
        if not re.search("(protect)|(defend)", sentence):
            continue
        
        moves = re.findall("(?:protect)|(?:defend).*?([A-Za-z]+[0-9]+)[ |\.]", explanation)
        moves = moves[:max_]
        if len(moves) == 0:
            continue  
            
        string = ", ".join(moves)
        
        # Needed because somtimes empty groups are captured
        if string == "":
            continue
        
        result = "This move defends " + string + "."
        return result
    return None

defense = [defenseExtractor(explanation) for explanation in explanations if defenseExtractor(explanation)]
len(defense)

268

In [224]:
def extract(explanation):
    transformed_explanations = []
    threat = threatExtractor(explanation)
    if threat:
        transformed_explanations.append(threat)
        
    mistake = mistakeExtractor(explanation)
    if mistake:
        transformed_explanations.append(mistake)
        
    attack = attackExtractor(explanation)
    if attack:
        transformed_explanations.append(attack)
        
    defense = defenseExtractor(explanation)
    if defense:
        transformed_explanations.append(defense)
        
    return transformed_explanations
        
counter = 0
for explanation in tqdm(explanations):
    if len(extract(explanation)) > 0:
        counter += 1

counter

100%|██████████| 17615/17615 [00:02<00:00, 7139.09it/s]


1450

In [225]:
extract(explanations[1])

['This move threatens d5.', 'This move defends Pe4.']

In [226]:
# Lets now rerun the model with the new captions
import re

def dataExtractor3(games):
    """
    Extracts the moves and fens of each game as strings
    path - the path to the pgn file in which the games are stored
    returns an array of objects containing the moves and the fens
    """
    
    counter = 0
    
    re0 = re.compile(r"{.*?}", re.MULTILINE)
    re1 = re.compile(r"{.*}", re.MULTILINE)
    re2 = re.compile(r"\d+\..", re.MULTILINE)
    re3 = re.compile(r"\.", re.MULTILINE)
    re4 = re.compile(r"\$\d+", re.MULTILINE)
    
    data = []
    for game in tqdm(games):
        result = [] # Stores all the commented fens for this game
        
        string = str(game.mainline_moves())
        string = re.sub(re0, "", string) # Remove comments
        string = re.sub(re1, "", string)
        string = re.sub(re2, "", string)
        string = re.sub(re3, "", string)
        string = re.sub(re4, "", string)
        
        if "{" in string:
            continue
            
        white = [name.strip(' ') for name in game.headers["White"].split(",")]
        black = [name.strip(' ') for name in game.headers["Black"].split(",")]
        
        moves = list(filter(None, string.split(" ")))
        mainline = list(game.mainline())
            
        board = game.board()                
        
        for i in range(len(moves)):
            move = moves[i]
            comment = mainline[i].comment
            board_prev = str(board.board_fen())
            board.push(mainline[i].move)
            board_after = str(board.board_fen())
            
            if comment: # comment exists  
                for name in white:
                    comment = comment.replace(name, 'white')
                for name in black:
                    comment = comment.replace(name, 'black')
                    
                comment = extract(comment)
                for c in comment:
                    counter += 1
                    result.append([[board_prev, board_after], c])
            
        data.append(result)
                               
    print(counter, 'commented moves found')        
    return data
                               
data = dataExtractor3(annotated_games)

100%|██████████| 3683/3683 [00:36<00:00, 100.21it/s]


1572 commented moves found


In [211]:
data[0][0]

[['r1bqkbnr/pp1ppppp/2n5/8/3NP3/8/PPP2PPP/RNBQKB1R',
  'r1bqkbnr/pp1p1ppp/2n5/4p3/3NP3/8/PPP2PPP/RNBQKB1R'],
 'This move threatens d5.']

In [212]:
vocab = {
    "<Pad>": 0,
    "<End>": 1
}

for game in tqdm(data):
    for fens, caption in game:
        words = caption.split(' ')
        
        for word in words:
            word = word.lower()
            if word not in vocab:
                vocab[word] = len(vocab)
                
print(len(vocab))

100%|██████████| 2207/2207 [00:00<00:00, 737889.91it/s]


423


In [213]:
data_preped = []
max_ = 0

for game in tqdm(data):
    for fens, caption in game:
        words = caption.split(' ')
        result = []
        
        for word in words:
            word = word.lower()            
            data_preped.append([fens, result, word])  
            result.append(word)
            
        max_ = max(max_, len(result) + 1)
            
        data_preped.append([fens, result, "<End>"])  
                
print("Data", len(data_preped))
print("Longest caption", max_)

100%|██████████| 2207/2207 [00:00<00:00, 456856.62it/s]


Data 7769
Longest caption 6


In [215]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, vocab):
        'Initialization'
        self.batch_size = 64
        self.data = data
        self.vocab = vocab

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        data = self.data[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(data)

        return X, y

    def __data_generation(self, data):
        'Generates data containing batch_size samples' 
        from_matrices = []
        to_matrices = []
        sentences = []
        targets = []

        for fens, sentence, t in data:
            from_matrix = fenToMatrix(fens[0])
            to_matrix = fenToMatrix(fens[1])
            
            from_matrices.append(from_matrix)
            to_matrices.append(to_matrix)
            
            sentence = [self.vocab[word] for word in sentence]
            
            target = np.zeros(len(self.vocab))
            target[self.vocab[t]] = 1
            sentence = sentence + [0] * (277 - len(sentence)) # Add padding

            sentences.append(sentence)
            targets.append(target)
            
        from_matrices = np.array(from_matrices)
        to_matrices = np.array(to_matrices)
        sentences = np.array(sentences)
        targets = np.array(targets)
            
        return [from_matrices, to_matrices, sentences], targets
    
np.random.shuffle(data_preped)
training_generator = DataGenerator(data_preped[:7000], vocab)
validation_generator = DataGenerator(data_preped[7000:], vocab)

Using TensorFlow backend.


In [216]:
training_generator[0][1][0].shape

(423,)

In [217]:
from keras.models import load_model

model = load_model('autoencoder1.h5')
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8, 8, 13)          0         
_________________________________________________________________
model_2 (Model)              (None, 256)               1509120   
_________________________________________________________________
model_1 (Model)              (None, 8, 8, 13)          1509696   
Total params: 3,018,816
Trainable params: 3,018,816
Non-trainable params: 0
_________________________________________________________________
None


In [218]:
from keras.models import Model
from keras.layers import Dense, Activation, Input, Flatten, Reshape, GRU, Embedding, concatenate
from keras import metrics

def Encoder():
    from_matrix = Input(shape=(8, 8, 13))
    to_matrix = Input(shape=(8, 8, 13))
    sentence = Input(shape=(10,))
    
    encoder = load_model('autoencoder1.h5').get_layer('model_2')
    encoder.name = 'encoder1'
    encoder.layers.pop(0)
    x1 = encoder(from_matrix)
    x1 = Model(inputs=from_matrix, outputs=x1)
    
    # Freeze the encoder
    for layer in x1.layers:
        layer.trainable=False
    
    encoder = load_model('autoencoder1.h5').get_layer('model_2')
    encoder.name = 'encoder2'
    encoder.layers.pop(0)
    x2 = encoder(to_matrix)
    x2 = Model(inputs=to_matrix, outputs=x2)
    
    # Freeze the encoder
    for layer in x2.layers:
        layer.trainable=False

    y = Embedding(len(vocab), 150, mask_zero=True)(sentence)
    y = GRU(124, activation='relu')(y)
    y = Model(inputs=sentence, outputs=y)
    
    z = concatenate([x1.output, x2.output, y.output])
    
    return Model(inputs=[x1.input, x2.input, y.input], outputs=z)

def Decoder():
    X = Input(shape=(636,))
    d1 = Dense(256, activation='relu')(X)
    d2 = Dense(len(vocab), activation='softmax')(d1)
    return Model(X, d2)


# define input to the model:
X = [Input(shape=(8, 8, 13)), Input(shape=(8, 8, 13)), Input(shape=(None,))]

# make the model:
explanation_model = Model(X, Decoder()(Encoder()(X)))

# compile the model:
explanation_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy])

In [219]:
explanation_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8, 8, 13)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 8, 8, 13)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 636)          3183990     input_1[0][0]                    
                                                                 input_2[0][0]                    
          

In [220]:
explanation_model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=5)

Epoch 1/5
109/109 [==============================] - ETA: 2:30 - loss: 6.3258 - categorical_accuracy: 0.0000e+ - ETA: 1:37 - loss: 6.0750 - categorical_accuracy: 0.0000e+ - ETA: 1:19 - loss: 5.8688 - categorical_accuracy: 0.0156   - ETA: 1:08 - loss: 5.5680 - categorical_accuracy: 0.05 - ETA: 1:02 - loss: 5.3420 - categorical_accuracy: 0.08 - ETA: 58s - loss: 5.1387 - categorical_accuracy: 0.0964 - ETA: 56s - loss: 4.9571 - categorical_accuracy: 0.109 - ETA: 53s - loss: 4.7129 - categorical_accuracy: 0.130 - ETA: 51s - loss: 4.6162 - categorical_accuracy: 0.137 - ETA: 49s - loss: 4.5288 - categorical_accuracy: 0.142 - ETA: 48s - loss: 4.4728 - categorical_accuracy: 0.147 - ETA: 47s - loss: 4.3625 - categorical_accuracy: 0.158 - ETA: 46s - loss: 4.3255 - categorical_accuracy: 0.158 - ETA: 44s - loss: 4.2703 - categorical_accuracy: 0.157 - ETA: 44s - loss: 4.2157 - categorical_accuracy: 0.164 - ETA: 43s - loss: 4.1667 - categorical_accuracy: 0.165 - ETA: 43s - loss: 4.1302 - categorical_

109/109 [==============================] - ETA: 48s - loss: 3.2838 - categorical_accuracy: 0.218 - ETA: 49s - loss: 3.0563 - categorical_accuracy: 0.210 - ETA: 49s - loss: 2.8869 - categorical_accuracy: 0.208 - ETA: 49s - loss: 2.8353 - categorical_accuracy: 0.195 - ETA: 49s - loss: 2.8286 - categorical_accuracy: 0.193 - ETA: 49s - loss: 2.8175 - categorical_accuracy: 0.187 - ETA: 50s - loss: 2.7789 - categorical_accuracy: 0.194 - ETA: 52s - loss: 2.8078 - categorical_accuracy: 0.191 - ETA: 52s - loss: 2.8138 - categorical_accuracy: 0.189 - ETA: 51s - loss: 2.7784 - categorical_accuracy: 0.196 - ETA: 49s - loss: 2.7463 - categorical_accuracy: 0.210 - ETA: 49s - loss: 2.7141 - categorical_accuracy: 0.210 - ETA: 48s - loss: 2.7089 - categorical_accuracy: 0.212 - ETA: 47s - loss: 2.6927 - categorical_accuracy: 0.210 - ETA: 47s - loss: 2.6799 - categorical_accuracy: 0.207 - ETA: 46s - loss: 2.6733 - categorical_accuracy: 0.209 - ETA: 45s - loss: 2.7039 - categorical_accuracy: 0.210 - ETA: 

109/109 [==============================] - ETA: 42s - loss: 2.8522 - categorical_accuracy: 0.171 - ETA: 41s - loss: 2.5778 - categorical_accuracy: 0.218 - ETA: 40s - loss: 2.6818 - categorical_accuracy: 0.203 - ETA: 42s - loss: 2.6033 - categorical_accuracy: 0.203 - ETA: 42s - loss: 2.5548 - categorical_accuracy: 0.221 - ETA: 42s - loss: 2.4628 - categorical_accuracy: 0.234 - ETA: 41s - loss: 2.4782 - categorical_accuracy: 0.218 - ETA: 41s - loss: 2.5119 - categorical_accuracy: 0.224 - ETA: 40s - loss: 2.5052 - categorical_accuracy: 0.220 - ETA: 39s - loss: 2.5009 - categorical_accuracy: 0.225 - ETA: 38s - loss: 2.4985 - categorical_accuracy: 0.217 - ETA: 40s - loss: 2.5276 - categorical_accuracy: 0.210 - ETA: 39s - loss: 2.5370 - categorical_accuracy: 0.211 - ETA: 39s - loss: 2.5313 - categorical_accuracy: 0.213 - ETA: 38s - loss: 2.5417 - categorical_accuracy: 0.213 - ETA: 38s - loss: 2.5405 - categorical_accuracy: 0.206 - ETA: 37s - loss: 2.5365 - categorical_accuracy: 0.208 - ETA: 

109/109 [==============================] - ETA: 47s - loss: 2.4373 - categorical_accuracy: 0.281 - ETA: 46s - loss: 2.2531 - categorical_accuracy: 0.234 - ETA: 45s - loss: 2.3910 - categorical_accuracy: 0.197 - ETA: 45s - loss: 2.4004 - categorical_accuracy: 0.203 - ETA: 43s - loss: 2.4657 - categorical_accuracy: 0.203 - ETA: 43s - loss: 2.4777 - categorical_accuracy: 0.190 - ETA: 42s - loss: 2.4617 - categorical_accuracy: 0.189 - ETA: 41s - loss: 2.4683 - categorical_accuracy: 0.197 - ETA: 41s - loss: 2.4342 - categorical_accuracy: 0.204 - ETA: 41s - loss: 2.4487 - categorical_accuracy: 0.195 - ETA: 40s - loss: 2.4452 - categorical_accuracy: 0.191 - ETA: 40s - loss: 2.4186 - categorical_accuracy: 0.188 - ETA: 40s - loss: 2.4233 - categorical_accuracy: 0.187 - ETA: 39s - loss: 2.4079 - categorical_accuracy: 0.193 - ETA: 39s - loss: 2.4055 - categorical_accuracy: 0.191 - ETA: 38s - loss: 2.4014 - categorical_accuracy: 0.192 - ETA: 38s - loss: 2.3969 - categorical_accuracy: 0.194 - ETA: 

109/109 [==============================] - ETA: 43s - loss: 2.1652 - categorical_accuracy: 0.125 - ETA: 44s - loss: 2.1678 - categorical_accuracy: 0.148 - ETA: 45s - loss: 2.1729 - categorical_accuracy: 0.192 - ETA: 45s - loss: 2.1584 - categorical_accuracy: 0.203 - ETA: 46s - loss: 2.2213 - categorical_accuracy: 0.196 - ETA: 45s - loss: 2.2069 - categorical_accuracy: 0.197 - ETA: 45s - loss: 2.2387 - categorical_accuracy: 0.200 - ETA: 44s - loss: 2.2590 - categorical_accuracy: 0.189 - ETA: 43s - loss: 2.2638 - categorical_accuracy: 0.184 - ETA: 43s - loss: 2.2696 - categorical_accuracy: 0.185 - ETA: 42s - loss: 2.2544 - categorical_accuracy: 0.193 - ETA: 42s - loss: 2.2636 - categorical_accuracy: 0.188 - ETA: 41s - loss: 2.2698 - categorical_accuracy: 0.189 - ETA: 41s - loss: 2.2628 - categorical_accuracy: 0.192 - ETA: 40s - loss: 2.2558 - categorical_accuracy: 0.194 - ETA: 40s - loss: 2.2571 - categorical_accuracy: 0.190 - ETA: 40s - loss: 2.2431 - categorical_accuracy: 0.187 - ETA: 